## Import Dependencies, Model, and Data

In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import pandas as pd
from google.colab import files

# Upload financial_classification_model.h5 and create model
files.upload_file('financial_classification_model.h5')
nn = tf.keras.models.load_model('financial_classification_model.h5')

# Inspect model
nn.summary()

Saved financial_classification_model.h5 to /content/financial_classification_model.h5
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3040      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5501 (21.49 KB)
Trainable params: 5501 (21.49 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Optimization attempt 1:
    -Same data preprocessing, Testing different node counts and number of hidden layers for nn_2

In [2]:
# Read in data to pandas df
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_value_counts = application_df['APPLICATION_TYPE'].value_counts()

# Choose a cutoff value and create a list of application types to be replaced
application_types_to_replace = list(app_value_counts[app_value_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

In [4]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
class_value_counts = application_df['CLASSIFICATION'].value_counts()

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_value_counts[class_value_counts < 500].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

In [5]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_df, drop_first=True)

# Split preprocessed data into features and target arrays
y = application_dummies['IS_SUCCESSFUL']
X = application_dummies.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [6]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler and scale the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to numpy arrays
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [7]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Third hidden layer
nn_2.add(tf.keras.layers.Dense(units=32, activation='relu'))

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               4864      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 15233 (59.50 KB)
Trainable params: 15233 (59.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# Compile the model
nn_2.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

# Train the model
nn_2.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 2ms/step - loss: 0.5680 - accuracy: 0.7213
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7277
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5511 - accuracy: 0.7306
Epoch 4/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5495 - accuracy: 0.7316
Epoch 5/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5484 - accuracy: 0.7325
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7327
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5466 - accuracy: 0.7333
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5458 - accuracy: 0.7338
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5460 - accuracy: 0.7338
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5453 - accuracy: 0.7346

In [9]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5527 - accuracy: 0.7304 - 481ms/epoch - 2ms/step
Loss: 0.5526910424232483, Accuracy: 0.7303789854049683


## nn_2 reached accuracy of .73 after 50 Epochs
    -the adjustments made were: Increased depth and width of model and increased training epochs
    -No significant gains were seen with these changes